In [1]:
import numpy as np

In [2]:
def rmetric(L, X, Y):
    #  L stacked list of 'L' pos. def. 2x2 untere-dreiecks-matrix(saved as (3*n)x1, L = ['L'[0,0],'L'[1,0],'L'[1,1])
    # 'X','Y' 2x2 untere-dreiecks-matrizen, saved as above
    
    # TODO: add area constant 
    diag = np.reciprocal(L**2)
    diag[list(range(1,len(L),3))] = 1
    
    # TODO: Concatenate with rot. matrix
    A = np.diag(diag) 
    
    return X.T@A@Y

In [3]:
rmetric(np.array([1,2,3,3,4,6,7,8,9]), np.array([1,2,3,3,4,6,7,8,9]),np.array([1,2,3,3,4,6,7,8,9]))

85

In [4]:
def rexp(L,X):
    mask = list(range(1,len(L),3))
    cmask = np.ones(L.shape).astype(bool)
    cmask[mask] = 0
    E = L
    E[mask] += X[mask]
    E[cmask] *= np.exp(X[cmask]*np.reciprocal(L[cmask]))
    
    return E

In [5]:
rexp(np.array([1,2,3,3,4,6,7,8,9]).astype(float), np.array([1,2,3,3,4,6,7,8,9]).astype(float))

array([ 2.71828183,  4.        ,  8.15484549,  8.15484549,  8.        ,
       16.30969097, 19.0279728 , 16.        , 24.46453646])

In [6]:
def rlog(L,X):
    mask = list(range(1,len(L),3))
    cmask = np.ones(L.shape).astype(bool)
    cmask[mask] = 0
    E = np.zeros(L.shape)
    E[mask] = X[mask] - L[mask]
    E[cmask] = L[cmask]*np.log(X[cmask]*np.reciprocal(L[cmask]))
    
    return E

In [10]:
def Dcinvers(L,X):
    mask1 = list(range(0, len(L), 3))
    mask2 = list(range(1, len(L), 3))
    mask3 = list(range(2, len(L), 3))
    D = 2*L*X
    D[mask3] += D[mask2]
    D[mask2] = X[mask1]*L[mask2] + L[mask1]*X[mask2]

    return D
    

In [11]:
Dcinvers(np.array([1,2,3,3,4,6,7,8,9]).astype(float), np.array([1,2,3,3,4,6,7,8,9]).astype(float))

array([  2.,   4.,  26.,  18.,  24., 104.,  98., 112., 290.])

In [17]:
def Cholesky(P):
    mask1 = list(range(0, len(P), 3))
    mask2 = list(range(1, len(P), 3))
    mask3 = list(range(2, len(P), 3))
    L = P
    L[mask1] = np.sqrt(P[mask1])
    L[mask2] = np.divide(P[mask2],L[mask1])
    L[mask3] = np.sqrt(P[mask3]- L[mask2]**2)
    
    return L

In [105]:
Cholesky(np.array([1,2,3]).astype(float))

/tmp/ipykernel_1565/1900903308.py:8: RuntimeWarning: invalid value encountered in sqrt
  L[mask3] = np.sqrt(P[mask3]- L[mask2]**2)


array([ 1.,  2., nan])

In [46]:
def inv_L(L):
    mask1 = list(range(0, len(L), 3))
    mask2 = list(range(1, len(L), 3))
    mask3 = list(range(2, len(L), 3))
    IL = L
    IL[mask1+mask3] = np.reciprocal(L[mask1+mask3])
    IL[mask2] = -L[mask2]*IL[mask1]*IL[mask3]
    
    return IL

In [48]:
inv_L(np.array([1,2,3,4,5,6]).astype(float))

array([ 1.        , -0.66666667,  0.33333333,  0.25      , -0.20833333,
        0.16666667])

In [78]:
def vector_to_matrix(vector, group = None):
    lengthy_v = np.zeros(int((len(vector)/3)*4))
    mask_without = [i for i in range(len(lengthy_v)) if i % 4 != 1]
    mask2 = list(range(1,len(lengthy_v),4))
    mask3 =list(range(2,len(lengthy_v),4))
    lengthy_v[mask_without] = vector
    if group == 'sym':
        lengthy_v[mask2] = lengthy_v[mask3]
        
    return np.reshape(lengthy_v,(-1,2,2), order = 'A')

In [75]:
def matrix_to_vector(matrix):
    lengthy_v = np.reshape(matrix, (-1), order = 'A')
    mask_without = [i for i in range(len(lengthy_v)) if i % 4 != 1]
    v = lengthy_v[mask_without]
    
    return v

In [76]:
a = vector_to_matrix(np.array([1,2,3,3,4,6,7,8,9]).astype(float))
b = a
print(a)
c = a @ b
print(c)

[[[1. 0.]
  [2. 3.]]

 [[3. 0.]
  [4. 6.]]

 [[7. 0.]
  [8. 9.]]]
[[[  1.   0.]
  [  8.   9.]]

 [[  9.   0.]
  [ 36.  36.]]

 [[ 49.   0.]
  [128.  81.]]]


In [103]:
def Dc(P,W):
    L = vector_to_matrix(Cholesky(P))
    print(Cholesky(P))
    Linv = np.linalg.inv(L)
    print(Linv)
    W = vector_to_matrix(W, group = 'sym')
    print(Linv.transpose(0,2,1).shape)
    inner = Linv@W@np.transpose(Linv, axes= (0,2,1))
    vecInner = matrix_to_vector(inner)
    mask1 = list(range(0, len(vecInner), 3))
    mask3 = list(range(2, len(vecInner), 3))
    vecInner[mask1+mask3] /= 2
    inner = vector_to_matrix(vecInner)
    
    D = L@inner

    return matrix_to_vector(D)

In [106]:
Dc(np.array([1,2,1,3,4,1,7,8,1]).astype(float), np.array([1,2,1,3,4,1,7,8,1]).astype(float))

[1.         2.                nan 1.31607401 1.75476535        nan
 1.62657656 1.85894464        nan]
[[[nan nan]
  [nan nan]]

 [[nan nan]
  [nan nan]]

 [[nan nan]
  [nan nan]]]
(3, 2, 2)


/tmp/ipykernel_1565/1900903308.py:8: RuntimeWarning: invalid value encountered in sqrt
  L[mask3] = np.sqrt(P[mask3]- L[mask2]**2)


array([nan, nan, nan, nan, nan, nan, nan, nan, nan])